In [1]:
DEBUG = False

### Customer preprocessing

In [2]:
import pandas as pd
from embeddings import TFIDFEmbedding, CharNgramEmbedding
import numpy as np

# Load raw customer data
raw_data_path = '../input_data/DATASET.xlsx'
customer_df = pd.read_excel(raw_data_path, sheet_name='customer')
# if DEBUG:
#     customer_df = customer_df[:100]  # For debugging, limit to first 100 rows

# Load street-district mapping
street_district_path = '../processed_data/street_district.csv'
street_district_df = pd.read_csv(street_district_path)
street_district_dict = street_district_df.set_index('street')['district'].to_dict()
street_district_combined = set(street_district_dict.keys()).union(set(street_district_dict.values()))

# Load precomputed TF-IDF embeddings
# embedding_model = TFIDFEmbedding('../processed_data/embedding.pkl')
embedding_model = CharNgramEmbedding('../processed_data/chargram_embedding.pkl')

# Display first rows of customer data
customer_df.head()

,customerid,DOB,gender,address,Website,job,industry
0,0000000014,36268,Nữ,hoa khe quan thanh khe,KH0104|0345,student,computer
1,0000000034,35901,Nam,man thai,KH0104|0333,student,health service
2,0000000051,34319,Nam,que son quang nam,KH0104|0255,blue collar,economics
3,0000000081,36472,Nam,truong chinh,KH0104|0293,student,economics
4,0000000098,34608,Nam,cẩm lệ,KH0104|40580,blue collar,health service


#### Query by vector to process address

In [3]:
# Make a copy of customer_df to new_customer_df so that the original remains unchanged
new_customer_df = customer_df.copy()

# Update the 'address' column in new_customer_df by matching customer addresses
new_customer_df['address'] = new_customer_df['address'].apply(
    lambda addr: embedding_model.query_in_initial_set(addr, 1)[0] if pd.notnull(addr) else None
)

new_customer_df['address'] = new_customer_df['address'].apply(
    lambda x: x if x in street_district_dict.values() else (street_district_dict[x] if x in street_district_dict else x)
)

# Display the updated new_customer_df head
new_customer_df.head()


,customerid,DOB,gender,address,Website,job,industry
0,0000000014,36268,Nữ,Thanh Khê,KH0104|0345,student,computer
1,0000000034,35901,Nam,Cẩm Lệ,KH0104|0333,student,health service
2,0000000051,34319,Nam,Thanh Khê,KH0104|0255,blue collar,economics
3,0000000081,36472,Nam,Hải Châu,KH0104|0293,student,economics
4,0000000098,34608,Nam,Cẩm Lệ,KH0104|40580,blue collar,health service


##### Process DOB field from MS Excel format to age

In [4]:
new_customer_df['DOB'] = pd.to_datetime(
    new_customer_df['DOB'],
    unit='D',
    origin='1900-01-01',
    errors='coerce'
)

current_date = pd.Timestamp('2019-01-01')

new_customer_df['age'] = new_customer_df['DOB'].apply(
    lambda dob: current_date.year - dob.year - ((current_date.month, current_date.day) < (dob.month, dob.day)) if pd.notnull(dob) else None
)
new_customer_df.head()


,customerid,DOB,gender,address,Website,job,industry,age
0,0000000014,1999-04-20,Nữ,Thanh Khê,KH0104|0345,student,computer,19.0
1,0000000034,1998-04-18,Nam,Cẩm Lệ,KH0104|0333,student,health service,20.0
2,0000000051,1993-12-18,Nam,Thanh Khê,KH0104|0255,blue collar,economics,25.0
3,0000000081,1999-11-10,Nam,Hải Châu,KH0104|0293,student,economics,19.0
4,0000000098,1994-10-03,Nam,Cẩm Lệ,KH0104|40580,blue collar,health service,24.0


In [5]:
new_customer_dict = new_customer_df.set_index('customerid').to_dict('index')
# new_customer_dict

### Crawled movies loading

In [6]:
crawled_movie_path = '../input_data/CrawledFilms.xlsx'
crawled_movie_df = pd.read_excel(crawled_movie_path)
crawled_movie_df.head()

,title,listed_in
0,DORAEMON: NOBITA VA MAT TRANG PHIEU LUU KY,"Animation,Action,Adventure,Comedy,Family,Fanta..."
1,ALADDIN,"Adventure,Comedy,Family,Fantasy,Musical,Romance"
2,AVENGERS: HOI KET,"Action,Adventure,Drama,Sci-Fi"
3,CA CHON ANH DUNG DI,Romance
4,CHUA TE GODZILLA: DE VUONG BAT TU,"Action,Adventure,Fantasy,Sci-Fi"


### Tickets preprocessing

In [7]:
tickets_df = pd.read_excel(raw_data_path, sheet_name='ticket')
if DEBUG:
    tickets_df = tickets_df[:100] 
tickets_df.head()

,orderid,cashier,saledate,total,customerid,ticketcode,date,time,slot,room,film,slot type,ticket type,ticket price,popcorn
0,10006052019B0225,emp002,2019-05-06 16:40:43,90000.0,0000000014,20074925,2019-05-06,18:15:00,G04,3,AVENGERS: HỒI KẾT,ĐƠN,Thành viên,45000,Không
1,10006052019B0225,emp002,2019-05-06 16:40:43,90000.0,0000000014,20074924,2019-05-06,18:15:00,G03,3,AVENGERS: HỒI KẾT,ĐƠN,Thành viên,45000,Không
2,10022052019B0167,emp011,2019-05-22 20:30:28,90000.0,0000000034,20097042,2019-05-22,20:45:00,E04,2,JOHN WICK 3: CHUẨN BỊ CHIẾN TRANH (C18),ĐƠN,Thành viên,45000,Không
3,10022052019B0167,emp011,2019-05-22 20:30:28,90000.0,0000000034,20097041,2019-05-22,20:45:00,E03,2,JOHN WICK 3: CHUẨN BỊ CHIẾN TRANH (C18),ĐƠN,Thành viên,45000,Không
4,10130052019B0141,emp005,2019-05-30 19:28:44,90000.0,0000000034,20106761,2019-05-30,19:30:00,F04,1,NGÔI ĐỀN KỲ QUÁI (C18),ĐƠN,Thành viên,45000,Không


In [8]:
# Create a DataFrame from new_customer_dict and merge it into tickets_df
customer_info_df = pd.DataFrame.from_dict(new_customer_dict, orient='index').reset_index().rename(columns={'index': 'customerid'})
tickets_df = tickets_df.merge(customer_info_df, on='customerid', how='left')
tickets_df.head()

,orderid,cashier,saledate,total,customerid,ticketcode,date,time,slot,room,...,ticket type,ticket price,popcorn,DOB,gender,address,Website,job,industry,age
0,10006052019B0225,emp002,2019-05-06 16:40:43,90000.0,0000000014,20074925,2019-05-06,18:15:00,G04,3,...,Thành viên,45000,Không,1999-04-20,Nữ,Thanh Khê,KH0104|0345,student,computer,19.0
1,10006052019B0225,emp002,2019-05-06 16:40:43,90000.0,0000000014,20074924,2019-05-06,18:15:00,G03,3,...,Thành viên,45000,Không,1999-04-20,Nữ,Thanh Khê,KH0104|0345,student,computer,19.0
2,10022052019B0167,emp011,2019-05-22 20:30:28,90000.0,0000000034,20097042,2019-05-22,20:45:00,E04,2,...,Thành viên,45000,Không,1998-04-18,Nam,Cẩm Lệ,KH0104|0333,student,health service,20.0
3,10022052019B0167,emp011,2019-05-22 20:30:28,90000.0,0000000034,20097041,2019-05-22,20:45:00,E03,2,...,Thành viên,45000,Không,1998-04-18,Nam,Cẩm Lệ,KH0104|0333,student,health service,20.0
4,10130052019B0141,emp005,2019-05-30 19:28:44,90000.0,0000000034,20106761,2019-05-30,19:30:00,F04,1,...,Thành viên,45000,Không,1998-04-18,Nam,Cẩm Lệ,KH0104|0333,student,health service,20.0


### Use dynamic programming to find the relationship between Crawled movies and tickets

In [12]:
from algorithms import StringComparison, UnicodeEncoder

# Normalize movie titles
crawled_movie_df['norm_title'] = crawled_movie_df['title'].apply(UnicodeEncoder.convert_from_unicode)
tickets_df['norm_film'] = tickets_df['film'].apply(UnicodeEncoder.convert_from_unicode)

# Create a dictionary for best matches
matched_titles = {}

for ticket_title in tickets_df['norm_film'].unique():
    best_match = None
    best_score = 0.0

    for crawled_title in crawled_movie_df['norm_title'].unique():
        comp = StringComparison(ticket_title, crawled_title)
        score = comp.similarity()

        if score > best_score:
            best_score = score
            best_match = crawled_title

    # Only select if similarity > 0.6 to avoid confusion
    if best_score > 0.6:
        matched_titles[ticket_title] = best_match
    else:
        matched_titles[ticket_title] = None

# Create a mapping from norm_title to genres (listed_in)
genre_mapping = crawled_movie_df.set_index('norm_title')['listed_in'].to_dict()

# Add matched title and genre columns to tickets_df
tickets_df['matched_title'] = tickets_df['norm_film'].map(matched_titles)
tickets_df['genres'] = tickets_df['matched_title'].map(lambda x: genre_mapping.get(x, None))

# Display the updated dataframe
tickets_df.head()

,orderid,cashier,saledate,total,customerid,ticketcode,date,time,slot,room,...,DOB,gender,address,Website,job,industry,age,norm_film,matched_title,genres
0,10006052019B0225,emp002,2019-05-06 16:40:43,90000.0,0000000014,20074925,2019-05-06,18:15:00,G04,3,...,1999-04-20,Nữ,Thanh Khê,KH0104|0345,student,computer,19.0,AVENGERS: HOI KET,AVENGERS: HOI KET,"Action,Adventure,Drama,Sci-Fi"
1,10006052019B0225,emp002,2019-05-06 16:40:43,90000.0,0000000014,20074924,2019-05-06,18:15:00,G03,3,...,1999-04-20,Nữ,Thanh Khê,KH0104|0345,student,computer,19.0,AVENGERS: HOI KET,AVENGERS: HOI KET,"Action,Adventure,Drama,Sci-Fi"
2,10022052019B0167,emp011,2019-05-22 20:30:28,90000.0,0000000034,20097042,2019-05-22,20:45:00,E04,2,...,1998-04-18,Nam,Cẩm Lệ,KH0104|0333,student,health service,20.0,JOHN WICK 3: CHUAN BI CHIEN TRANH (C18),JOHN WICK 3: CHUAN BI CHIEN TRANH,"Action,Crime,Thriller"
3,10022052019B0167,emp011,2019-05-22 20:30:28,90000.0,0000000034,20097041,2019-05-22,20:45:00,E03,2,...,1998-04-18,Nam,Cẩm Lệ,KH0104|0333,student,health service,20.0,JOHN WICK 3: CHUAN BI CHIEN TRANH (C18),JOHN WICK 3: CHUAN BI CHIEN TRANH,"Action,Crime,Thriller"
4,10130052019B0141,emp005,2019-05-30 19:28:44,90000.0,0000000034,20106761,2019-05-30,19:30:00,F04,1,...,1998-04-18,Nam,Cẩm Lệ,KH0104|0333,student,health service,20.0,NGOI ĐEN KY QUAI (C18),NGOI DEN KY QUAI,"Comedy,Horror"


In [13]:
tickets_df.to_csv('../processed_data/cleaned_raw_data.csv', index=False)